In [ ]:
import scipy as sp
import numpy as np
import scipy.sparse as sparse
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
from common import set_figure

import pyamg

###  Anisotropy

For this example, let's try a fairly small grid.

And test $\varepsilon = 0.01$ with a rotation of $\theta = \frac{\pi}{2}$ and $\theta=\frac{\pi}{4}$.

In [ ]:
#theta = np.pi / 2
theta = np.pi / 4
epsilon = 0.01
nx = 20
ny = nx
stencil = pyamg.gallery.diffusion.diffusion_stencil_2d(type='FD', epsilon=epsilon, theta=theta)
print(stencil)
A = pyamg.gallery.stencil_grid(stencil, (nx, ny), format='csr')

###  Build a CF hierarchy

Use `ruge_stuben_solver` to build a CF hierarchy. With a few notes:

- `keep` will simply retain all of the "extra" operators used to build the hierarch.  For example a list of the C/F nodes on each level.
- `strength` sets the strength parameters.  Here `classical` is normally used.  The `norm` equal to `min` (rather than `abs`) avoids using the absolute value in the strength test so that large positive entries are treated as weak connections.

In [ ]:
ml = pyamg.ruge_stuben_solver(A, keep=True,
                              strength=('classical',
                                        {'theta': 0.25,
                                         'norm': 'min'}))
print(ml)

### View the splitting

We can vew the C/F splitting by inspecting the first `level`.

In [ ]:
X, Y = np.meshgrid(np.linspace(0,1,nx), np.linspace(0,1,ny))
X = X.ravel()
Y = Y.ravel()

# The CF splitting, 1 == C-node and 0 == F-node
splitting = ml.levels[0].splitting
C_nodes = splitting == 1
F_nodes = splitting == 0

plt.scatter(X[C_nodes], Y[C_nodes],
            marker='s',
            s=30.0, label='C pts')
plt.scatter(X[F_nodes], Y[F_nodes],
            marker='s',
            s=30.0, label='F pts')
plt.legend(frameon=True)

### The `ml` hierarchy

Notice a few things about the hierarchy...

Each level has a number of attributes/functions:
- `A` is the operator on this level
- `P` interpolates **to** this level
- `R` restricts **from** this level.  Simply `R=P.T` in this case.
- `C` is a sparse matrix of strong connections
- `splitting` holds C/F information
- `presmoother` and `postsmoother` point to the relaxation routine

In [ ]:
dir(ml.levels[0])

- `Operator complexity` is the total sum of the `nnz` in all operators `A` compared to the fine level

In [ ]:
Asizes = [ml.levels[i].A.nnz for i in range(len(ml.levels))]
print(Asizes)
print(np.sum(Asizes)/Asizes[0])

- `Grid Complexity` is the total sum of the unknowns per level, compared to the fine level

In [ ]:
Asizes = [ml.levels[i].A.shape[0] for i in range(len(ml.levels))]
print(Asizes)
print(np.sum(Asizes)/Asizes[0])

### Set up a problem and solve

Now, set $f=0$ and the initial $u$ to random and solve.

Note: you can set `accel='cg'` to use AMG as a preconditioner

In [ ]:
f = np.zeros(A.shape[0])
u0 = np.random.randn(A.shape[0])
res = []
u = ml.solve(b=f, x0=u0, residuals=res)

In [ ]:
res = np.array(res)
res[1:] / res[:-1]

### Make a bigger problem!

In [ ]:
theta = np.pi / 4
epsilon = 0.01
nx = 1000
ny = nx
stencil = pyamg.gallery.diffusion.diffusion_stencil_2d(type='FD', epsilon=epsilon, theta=theta)
print(stencil)
A = pyamg.gallery.stencil_grid(stencil, (nx, ny), format='csr')

In [ ]:
ml = pyamg.ruge_stuben_solver(A, keep=True,
                              strength=('classical',
                                        {'theta': 0.25,
                                         'norm': 'min'}))
print(ml)

In [ ]:
f = np.zeros(A.shape[0])
u0 = np.random.randn(A.shape[0])
res = []
u = ml.solve(b=f, x0=u0, residuals=res, accel='cg')

In [ ]:
res = np.array(res)
res[1:] / res[:-1]

In [ ]:
plt.semilogy(res)